# Toy-dataset Generator

In [22]:
# Codewords a , b, c, d, e, f

# forms (3-5) A (1-2) B (3-5) C

# specific forms abc, dbe, afc, afe, cbd, eba, cfd, efa

# z dimension : 8 with variance around parameters

# Target result: feature embeddings into 2 dimensions produce mappings b ~= f, a ~= d, c ~= e
%reload_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'/scratch/Jack/projects/Chamber')
sys.path.insert(0,'/scratch/Jack/projects/Explanations')

import random
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from models import VAE, ConvVAE, W2V, Analogy
from chamber import Chamber, Misc, Historian, Oracle, Commander

def generate_sequence(n=1):
    p = []
    for i in range(n):
        random_pairs = [('a','b'),('b','b'),('c','b'),('a','d'),('c','d'),('d','d')]
        # a and b are equivalent, but different from c and d
        pi = random.choice(random_pairs)
        for pi in random.choice(random_pairs):
            p += [generate_latent(pi) for x in range(random.randint(1,1))]
        #p += [generate_latent(p1) for x in range(random.randint(1,1))]+[generate_latent(p2) for x in range(random.randint(1,1))]+[generate_latent(p3) for x in range(random.randint(1,1))] 
    
    return p

def generate_latent(c, one_hot = True):
    if one_hot:
        t = torch.Tensor([0 for i in range(6)])
        t[ord(c)-ord('a')] = 1
        return [t]+[ord(c)-ord('a')]
    else:
        if c == 'a':
            return [torch.Tensor([1.0, 1.0])]+[ord(c)-ord('a')]
        if c == 'b':
            return [torch.Tensor([0.0, 0.0])]+[ord(c)-ord('a')]
        if c == 'c':
            return [torch.Tensor([1.0, 0.0])]+[ord(c)-ord('a')]
        if c == 'd':
            return [torch.Tensor([0.0 ,1.0])]+[ord(c)-ord('a')]
        
epochs = 5000
near_window = 1
back_window = False


m = torch.nn.LogSoftmax()
loss = torch.nn.NLLLoss().cuda()
optimizer = torch.optim.Adam(w2v.parameters(), weight_decay=0, lr=0.01)

# So the training over latent space doesn't work as well as it should
# Now we use the analogy model instead on train_analogy
def train(word_pair):
    in_word = Variable(word_pair[0][0].cuda()).unsqueeze(0)
    out_word = Variable(torch.LongTensor([word_pair[1][1]]).cuda()).long()
    
    # categorical out_word
    out_pred = w2v(in_word)
    
    
    myloss = loss(m(out_pred[1]), out_word)
    n = m(out_pred[1]).exp().data.cpu().numpy()[0]
    #print(myloss.data.cpu().numpy()[0])
    #for el in n:
    #    print("{:0.2f}".format(float(el)),end=" , "), np.around(n,decimals=2)
    #print("")
    #print(out_word.data.cpu().numpy()[0], in_word.data.cpu().numpy()[0], out_pred[0].data.cpu().numpy()[0])

    optimizer.zero_grad()
    myloss.backward()
    optimizer.step()


loss_analogy = torch.nn.MSELoss().cuda()
dist = torch.nn.PairwiseDistance().cuda()
optimizer_analogy = torch.optim.Adam(analogy.parameters(), weight_decay=0, lr=0.001)
def train_analogy(word_pair):
    in_word = Variable(word_pair[0][0].cuda()).unsqueeze(0)
    out_word = Variable(word_pair[1][0].cuda()).unsqueeze(0)
    
    in_embedding = analogy.embed(in_word)
    out_embedding = analogy.embed(out_word)
    # categorical out_word
    out_pred = analogy(in_embedding)
    
    distance = dist(out_pred[1], out_embedding)
    myloss = loss_analogy(distance, Variable(torch.Tensor([0.0]).cuda()))
    
    #print(myloss.data.cpu().numpy()[0])
    
    optimizer_analogy.zero_grad()
    myloss.backward()
    optimizer_analogy.step()
    
experiments = 1

w2v = W2V(dict_size=6, hidden_features=2).cuda()
analogy = Analogy().cuda()

for experiment in range(experiments):
    
    for i in range(epochs):
        s = generate_sequence()
        for word in range(len(s)):
            for l in range(-near_window,near_window+1):
                if l == 0:
                    continue

                if l < 0 and back_window == False:
                    continue

                word_near = word-l
                if word_near < 0 or word_near >= len(s):
                    continue
                train([s[word], s[word_near]])
                #print(s[word][1], s[word_near][1])
                #train_analogy([s[word], s[word_near]])

    print("\nExperiment\n\n")
    for var in ['a','b','c','d']:
        in_word = Variable(generate_latent(var)[0].cuda()).unsqueeze(0)
        embed = w2v(in_word)
        print(var, embed[0].data.cpu().numpy()[0], in_word.data.cpu().numpy()[0])


Experiment


a [-0.00222291  0.00859151] [ 1.  0.  0.  0.  0.  0.]
b [ 0.00957477 -0.01385114] [ 0.  1.  0.  0.  0.  0.]
c [ 0.01293331  0.00684991] [ 0.  0.  1.  0.  0.  0.]
d [-0.00456683  0.00329325] [ 0.  0.  0.  1.  0.  0.]
